In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

In [2]:
# Create Tensors to hold input and outputs.
beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=2

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs_1):
        lambda1,lambda2,lr,ga= config["lambda1"], config["lambda2"],config["lr"],config["ga"]
        model_1=MLP(seed=1)
        model_2=MLP(seed=2)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1, layer1_out_1, layer2_out_1= model_1(inputs_1[idx_train])
            out_2, layer1_out_2, layer2_out_2= model_2(inputs_2[idx_train])
            out_3, layer1_out_3, layer2_out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params = model_1.linear1
            model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
            model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
           

            model_2_all_linear1_params = model_1.linear1
            model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
            model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
    

            model_3_all_linear1_params = model_3.linear1
            model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
            model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            all_linear23_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params,
                                           model_2_all_linear2_params,model_2_all_linear3_params,
                                           model_3_all_linear2_params,model_3_all_linear3_params),0)

            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out_1, test_layer1_out_1, test_layer2_out_1= model_1(train_x1[idx_test])
        test_out_2, test_layer1_out_2, test_layer2_out_2= model_2(train_x2[idx_test])
        test_out_3, test_layer1_out_3, test_layer2_out_3= model_3(train_x3[idx_test])
        
        test_BCE_loss_1 = loss_fn(test_out_1, targets_1[idx_test].reshape(-1,1))
        test_BCE_loss_2 = loss_fn(test_out_2, targets_2[idx_test].reshape(-1,1))
        test_BCE_loss_3 = loss_fn(test_out_3, targets_3[idx_test].reshape(-1,1))
        
        test_BCE_loss=test_BCE_loss_1+test_BCE_loss_2+test_BCE_loss_3
        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

In [4]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lambda2,eta,lr= config["lambda1"], config["lambda2"],config["eta"],config["lr"]
    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3
    
    
    sum_tGM=0
    for idx_train,idx_test in kf.split(inputs_1):
       
        
        model_1=MLP(seed=1)
        model_2=MLP(seed=1)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1[idx_train])
            out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2[idx_train])
            out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params = model_1.linear1
            model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
            model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
            model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])
           

            model_2_all_linear1_params = model_1.linear1
            model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
            model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
            model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])
    

            model_3_all_linear1_params = model_3.linear1
            model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
            model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
            model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                             model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                             model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear234_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out_1, test_layer1_out_1, test_layer2_out_1,test_layer3_out_1= model_1(train_x1[idx_test])
        test_out_2, test_layer1_out_2, test_layer2_out_2,test_layer3_out_2= model_2(train_x2[idx_test])
        test_out_3, test_layer1_out_3, test_layer2_out_3,test_layer3_out_3= model_3(train_x3[idx_test])
        

        test_prediction_1= (model_1(train_x1[idx_test])[0]>0.5).clone().int()
        test_target_1= train_y1[idx_test].reshape(-1,1).int()

        test_prediction_2= (model_2(train_x2[idx_test])[0]>0.5).clone().int()
        test_target_2= train_y2[idx_test].reshape(-1,1).int()

        test_prediction_3= (model_3(train_x3[idx_test])[0]>0.5).clone().int()
        test_target_3= train_y3[idx_test].reshape(-1,1).int()

        test_prediction=np.append(test_prediction_1,test_prediction_2)
        test_prediction=np.append(test_prediction,test_prediction_3)
        test_prediction=test_prediction.tolist()

        test_target=np.append(test_target_1,test_target_2)
        test_target=np.append(test_target,test_target_3)
        test_target=test_target.tolist()

        final_matrix=confusion_matrix(test_target,test_prediction)
        tTPR=recall_score(test_target,test_target)#TPR
        tTNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
        tGM=math.sqrt(tTPR*tTNR)

        sum_tGM+=tGM

    tune.report(my_test_sum_tGM=sum_tGM/split_num)

In [5]:
final_eta=[]
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [ ]:
for i in range(10):
    setup_seed(i)
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=2
# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            out= torch.sigmoid(self.linear3(layer2_out))
            return out, layer1_out, layer2_out


    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lambda2,lr,ga=final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
    model_1=MLP(seed=1)
    model_2=MLP(seed=2)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1, layer1_out_1, layer2_out_1= model_1(inputs_1)
        out_2, layer1_out_2, layer2_out_2= model_2(inputs_2)
        out_3, layer1_out_3, layer2_out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_1_all_linear1_params = model_1.linear1
        model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
        model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])


        model_2_all_linear1_params = model_1.linear1
        model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
        model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])


        model_3_all_linear1_params = model_3.linear1
        model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
        model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
        all_linear23_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, 
                                         model_2_all_linear2_params,model_2_all_linear3_params,
                                         model_3_all_linear2_params,model_3_all_linear3_params),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))

    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear23_params.pow(2))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
################## Y_prior##################    
    y_prior_1= (model_1(train_x1)[0]).detach().squeeze()

    y_prior_2= (model_2(train_x2)[0]).detach().squeeze()

    y_prior_3= (model_3(train_x3)[0]).detach().squeeze()
    
    
################## third step##################

    #no prior
    prior=0
    # choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)
        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out

    config={
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.002,0.003]),
             "lr": tune.choice([0.7,0.8,0.9]),
            "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
        }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=224)
    

################## fourth step##################

    #no prior
    prior=0
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    lambda1,lambda2,eta,lr=final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out



    model_1=MLP(seed=1)
    model_2=MLP(seed=1)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    for t in range(max_iteration):

        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1)
        out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2)
        out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_1_all_linear1_params = model_1.linear1
        model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
        model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
        model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])

        model_2_all_linear1_params = model_1.linear1
        model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
        model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
        model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])

        model_3_all_linear1_params = model_3.linear1
        model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
        model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
        model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
        all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                         model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                         model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))

    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear234_params.pow(2))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
####################################   prediction ####################################      

    prediction_1= (model_1(test_x1)[0]>0.5).clone().int()
    target_1= test_y1.reshape(-1,1).int()

    prediction_2= (model_2(test_x2)[0]>0.5).clone().int()
    target_2= test_y2.reshape(-1,1).int()

    prediction_3= (model_3(test_x3)[0]>0.5).clone().int()
    target_3= test_y3.reshape(-1,1).int()

    prediction=np.append(prediction_1,prediction_2)
    prediction=np.append(prediction,prediction_3)
    prediction=prediction.tolist()

    target=np.append(target_1,target_2)
    target=np.append(target,target_3)
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))
    
    
####################################   variable selection ####################################     
    model_1_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_2_weight=model_2_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_3_weight=model_3_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.1
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=cubic_root(model_1_weight[0,j]*model_2_weight[0,j]*model_3_weight[0,j])

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_1_weight[0,j]=0
            model_2_weight[0,j]=0
            model_3_weight[0,j]=0
        else:
            model_1_weight[0,j]=model_1_weight[0,j]
            model_2_weight[0,j]=model_2_weight[0,j]
            model_3_weight[0,j]=model_3_weight[0,j]
    
    index=np.nonzero(model_1_weight)[1]
    
    vs[:,index]+=1
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    TP=len(trueindex.intersection(index))
    FP=len(np.setdiff1d(index,trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    final_eta.append(eta)
    
    print(i)
    

2022-11-03 16:14:51,977	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8277 
2022-11-03 16:14:52,885	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_97ff5_00000,TERMINATED,10.224.0.12:20216,0.3,0.03,0.003,0.8,1,12.7066,1.45992
train_model_97ff5_00001,TERMINATED,10.224.0.12:20244,0.5,0.03,0.001,0.8,1,12.2819,1.42718
train_model_97ff5_00002,TERMINATED,10.224.0.12:20246,0.4,0.05,0.001,0.6,1,11.3491,1.34887
train_model_97ff5_00003,TERMINATED,10.224.0.12:20248,0.4,0.05,0.002,0.8,1,12.3912,1.17462
train_model_97ff5_00004,TERMINATED,10.224.0.12:20250,0.4,0.04,0.002,0.7,1,11.3844,1.3315
train_model_97ff5_00005,TERMINATED,10.224.0.12:20252,0.4,0.03,0.002,0.8,1,11.4911,1.43109
train_model_97ff5_00006,TERMINATED,10.224.0.12:20254,0.4,0.04,0.002,0.7,1,11.4912,1.3315
train_model_97ff5_00007,TERMINATED,10.224.0.12:20255,0.4,0.03,0.001,0.7,1,11.4247,1.62531
train_model_97ff5_00008,TERMINATED,10.224.0.12:20258,0.5,0.03,0.003,0.8,1,12.3424,1.2918
train_model_97ff5_00009,TERMINATED,10.224.0.12:20259,0.5,0.03,0.001,0.6,1,12.4254,1.63232


(raylet) [2022-11-03 16:15:01,866 E 19307 19325] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-03_16-14-49_940565_19181 is over 95% full, available space: 113442816; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_97ff5_00000:
  date: 2022-11-03_16-15-07
  done: false
  experiment_id: 1636f1ab7d3c48e395b447cfc9878ba5
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.459917378425598
  node_ip: 10.224.0.12
  pid: 20216
  time_since_restore: 12.706600427627563
  time_this_iter_s: 12.706600427627563
  time_total_s: 12.706600427627563
  timestamp: 1667463307
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 97ff5_00000
  warmup_time: 0.004024505615234375
  
Result for train_model_97ff5_00000:
  date: 2022-11-03_16-15-07
  done: true
  experiment_id: 1636f1ab7d3c48e395b447cfc9878ba5
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.459917378425598
  node_ip: 10.224.0.12
  pid: 20216
  time_since_restore: 12.706600427627563
  time_this_iter_s: 12.706600427627563
  time_total_s: 12.706600427627563
  timestamp: 1667463307
  timeste

Result for train_model_97ff5_00016:
  date: 2022-11-03_16-15-10
  done: true
  experiment_id: cae24d5a4b234203b35b273d7c9566fd
  experiment_tag: 16_ga=0.4000,lambda1=0.0300,lambda2=0.0030,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.4863402843475342
  node_ip: 10.224.0.12
  pid: 20273
  time_since_restore: 11.722229480743408
  time_this_iter_s: 11.722229480743408
  time_total_s: 11.722229480743408
  timestamp: 1667463310
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 97ff5_00016
  warmup_time: 0.004705190658569336
  
Result for train_model_97ff5_00017:
  date: 2022-11-03_16-15-10
  done: false
  experiment_id: 2b1b6bfffcf54385a9bb5b209006c765
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.2857456922531127
  node_ip: 10.224.0.12
  pid: 20275
  time_since_restore: 11.33513069152832
  time_this_iter_s: 11.33513069152832
  time_total_s: 11.33513069152832
  timestamp: 1667463310
  timeste

...

Result for train_model_d7198_00000:
  date: 2022-11-03_16-31-13
  done: false
  experiment_id: ab5af83e51b542a5b49486ad073934ca
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.2653082251548766
  node_ip: 10.224.0.12
  pid: 15439
  time_since_restore: 12.844937562942505
  time_this_iter_s: 12.844937562942505
  time_total_s: 12.844937562942505
  timestamp: 1667464273
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d7198_00000
  warmup_time: 0.0036933422088623047
  
Result for train_model_d7198_00000:
  date: 2022-11-03_16-31-13
  done: true
  experiment_id: ab5af83e51b542a5b49486ad073934ca
  experiment_tag: 0_ga=0.5000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.2653082251548766
  node_ip: 10.224.0.12
  pid: 15439
  time_since_restore: 12.844937562942505
  time_this_iter_s: 12.844937562942505
  time_total_s: 12.844937562942505
  timestamp: 1667464273
  time

(raylet) [2022-11-03 16:31:14,689 E 19307 19325] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-03_16-14-49_940565_19181 is over 95% full, available space: 13103104; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_d7198_00016:
  date: 2022-11-03_16-31-14
  done: false
  experiment_id: 3eb55c2fa05941a3894baf1b241bf826
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.7910606384277343
  node_ip: 10.224.0.12
  pid: 15997
  time_since_restore: 11.286123752593994
  time_this_iter_s: 11.286123752593994
  time_total_s: 11.286123752593994
  timestamp: 1667464274
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d7198_00016
  warmup_time: 0.007572174072265625
  
Result for train_model_d7198_00016:
  date: 2022-11-03_16-31-14
  done: true
  experiment_id: 3eb55c2fa05941a3894baf1b241bf826
  experiment_tag: 16_ga=0.3000,lambda1=0.0400,lambda2=0.0010,lr=0.6000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.7910606384277343
  node_ip: 10.224.0.12
  pid: 15997
  time_since_restore: 11.286123752593994
  time_this_iter_s: 11.286123752593994
  time_total_s: 11.286123752593994
  timestamp: 1667464274
  time

Result for train_model_d7198_00021:
  date: 2022-11-03_16-31-15
  done: true
  experiment_id: 4b463410ec41473ab7c2dbdb31919ba3
  experiment_tag: 21_ga=0.5000,lambda1=0.0300,lambda2=0.0020,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.133636474609375
  node_ip: 10.224.0.12
  pid: 16006
  time_since_restore: 11.771777629852295
  time_this_iter_s: 11.771777629852295
  time_total_s: 11.771777629852295
  timestamp: 1667464275
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d7198_00021
  warmup_time: 0.0048367977142333984
  
Result for train_model_d7198_00025:
  date: 2022-11-03_16-31-15
  done: false
  experiment_id: 3c40841ece6a46a9beb38238d5656084
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.5294184684753418
  node_ip: 10.224.0.12
  pid: 16014
  time_since_restore: 11.617854118347168
  time_this_iter_s: 11.617854118347168
  time_total_s: 11.617854118347168
  timestamp: 1667464275
  time

Result for train_model_d7198_00006:
  date: 2022-11-03_16-31-16
  done: false
  experiment_id: ef4c321247d54528b0cbec5a75faad64
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.937556838989258
  node_ip: 10.224.0.12
  pid: 15975
  time_since_restore: 12.448835372924805
  time_this_iter_s: 12.448835372924805
  time_total_s: 12.448835372924805
  timestamp: 1667464276
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d7198_00006
  warmup_time: 0.003397703170776367
  
Result for train_model_d7198_00026:
  date: 2022-11-03_16-31-16
  done: false
  experiment_id: fa2b300e68b84af1832fa07aec70e320
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.0302847027778625
  node_ip: 10.224.0.12
  pid: 16016
  time_since_restore: 12.092440128326416
  time_this_iter_s: 12.092440128326416
  time_total_s: 12.092440128326416
  timestamp: 1667464276
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d7198_00026


In [7]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.8771829594861206

0.8521361543850474

0.8639987171999562

0.8641666666666665

0.8560247695039832

0.8653834336798625

(raylet) [2022-10-18 20:03:18,842 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 668864512; capacity: 29394726912. Object creation will fail if spilling is required.


In [8]:
i

9

In [9]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

0.97

0.8833333333333334

0.9250242323057073

0.10800000000000001

0.8920000000000001

...

(raylet) [2022-10-18 22:48:56,368 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 509161472; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:49:06,376 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 508956672; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:49:16,384 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 508825600; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:49:26,391 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 508690432; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-18 22:54:06,603 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 503980032; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:54:16,610 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 503824384; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:54:26,618 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 503709696; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:54:36,625 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 503508992; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-18 22:59:16,830 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 498851840; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:59:26,837 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 498724864; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:59:36,844 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 498499584; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 22:59:46,851 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 498327552; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-18 23:04:27,052 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 493780992; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 23:04:37,058 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 493559808; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 23:04:47,065 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 493404160; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 23:04:57,072 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 493244416; capacity: 29394726912. Object creation will fail i

(raylet) [2022-10-18 23:09:37,274 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 488587264; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 23:09:47,281 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 488407040; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 23:09:57,289 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 488275968; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-10-18 23:10:07,296 E 32189 32207] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-18_19-39-00_802715_29775 is over 95% full, available space: 488079360; capacity: 29394726912. Object creation will fail i

In [11]:
vs

array([[5., 5., 5., 5., 5., 5., 4., 5., 5., 5., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0.]])